In [3]:
import json
import yfinance as yf
import pandas as pd
import numpy as np

In [6]:
class PortfolioAnalyzer:
    def __init__(self, stocks, weights, start_date, end_date, initial_investment):
        self.stocks = stocks 
        self.weights = weights
        self.start_date = start_date
        self.end_date = end_date
        self.initial_investment = initial_investment
        
    def convert_data(self, data, round_digit=2):
        result = []
        for col in data.columns:
            for index, row in data.iterrows():
                result.append({
                    "index": index,
                    "ticker": col,
                    "value": round(row[col], round_digit)
                })
        return json.dumps(result, indent=4)
        
    def download_price_data(self, convert_data=True):
        data = yf.download(self.stocks, start=self.start_date,
                           end=self.end_date)['Close']
        # Convert datetime index to string dates (YYYY-MM-DD) for JSON serialization
        data.index = [str(x.date()) for x in data.index]
        return self.convert_data(data) if convert_data else data

In [38]:
price_data = yf.download(['AAPL', 'MSFT'], start="2023-01-01", end="2023-12-31")['Close']
price_data

weights = [0.5, 0.5]

daily_returns = price_data.pct_change().dropna()
cumprod_returns = (daily_returns+1).cumprod()
initial_allocations = {ticker: 10000 * weight
                       for ticker, weight in zip(['AAPL', 'MSFT'], weights)}
init_series = pd.Series(initial_allocations)
asset_values = cumprod_returns.multiply(init_series, axis=1)
asset_values['Portfolio Value'] = asset_values.sum(axis=1)
asset_values.index = pd.to_datetime(asset_values.index)
asset_values

overall_moves_7d = {}
overall_moves_7d['ticker'] = []
overall_moves_7d['7d_change'] = []
ranking=3

for x in asset_values.columns:
    if x == 'Portfolio Value':
        continue
    
    overall_moves_7d['ticker'] += [x]
    overall_moves_7d['7d_change'] += [
        round(100*(asset_values[x].iloc[-1] - asset_values[x].iloc[-7])/asset_values[x].iloc[-7], 2)]

overall_moves_7d
print("winners",{"tickers": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change",ascending=False).iloc[:ranking]['ticker'].values),
"7d_change": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change",ascending=False).iloc[:ranking]['7d_change'].values)
        })
print("losers", {"tickers": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change", ascending=True).iloc[:ranking]['ticker'].values),
                 "7d_change": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change", ascending=True).iloc[:ranking]['7d_change'].values)
                 })

[*********************100%***********************]  2 of 2 completed

winners {'tickers': ['MSFT', 'AAPL'], '7d_change': [np.float64(1.46), np.float64(-1.18)]}
losers {'tickers': ['AAPL', 'MSFT'], '7d_change': [np.float64(-1.18), np.float64(1.46)]}


In [51]:
analyzer = PortfolioAnalyzer(['AAPL', 'MSFT'], [0.5, 0.5], '2023-01-01', '2023-12-31', 10000)
price_data = analyzer.download_price_data(convert_data=False)

daily_returns = price_data.pct_change().dropna()
cumprod_returns = (daily_returns+1).cumprod()
initial_allocations = {ticker: analyzer.initial_investment * weight
                       for ticker, weight in zip(analyzer.stocks, analyzer.weights)}
init_series = pd.Series(initial_allocations)
asset_values = cumprod_returns.multiply(init_series, axis=1)
asset_values['Portfolio Value'] = asset_values.sum(axis=1)
asset_values.index = pd.to_datetime(asset_values.index)
asset_values

overall_moves_7d = {}
overall_moves_7d['ticker'] = []
overall_moves_7d['7d_change'] = []
ranking = 3

for x in asset_values.columns:
    if x == 'Portfolio Value':
        continue

    overall_moves_7d['ticker'] += [x]
    overall_moves_7d['7d_change'] += [
        round(100*(asset_values[x].iloc[-1] - asset_values[x].iloc[-7])/asset_values[x].iloc[-7], 2)]

overall_moves_7d
# print("winners", {"tickers": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change", ascending=False).iloc[:ranking]['ticker'].values),
#                   "7d_change": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change", ascending=False).iloc[:ranking]['7d_change'].values)
#                   })
# print("losers", {"tickers": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change", ascending=True).iloc[:ranking]['ticker'].values),
#                  "7d_change": list(pd.DataFrame(overall_moves_7d).sort_values(by="7d_change", ascending=True).iloc[:ranking]['7d_change'].values)
#                  })

[*********************100%***********************]  2 of 2 completed


{'ticker': ['AAPL', 'MSFT'],
 '7d_change': [np.float64(-1.18), np.float64(1.46)]}

In [67]:
analyzer = PortfolioAnalyzer(
    ['SPEM', 'IEFA', 'BBRE', 'IVV', 'BIV'], [0.2247, 0.225, 0.075, 0.23461, 0.24069], '2023-01-01', '2023-12-31', 10000)
price_data = analyzer.download_price_data(convert_data=False)

daily_returns = price_data.pct_change().dropna()
cumprod_returns = (daily_returns+1).cumprod()
initial_allocations = {ticker: analyzer.initial_investment * weight
                       for ticker, weight in zip(analyzer.stocks, analyzer.weights)}
init_series = pd.Series(initial_allocations)
asset_values = cumprod_returns.multiply(init_series, axis=1)
asset_values['Portfolio Value'] = asset_values.sum(axis=1)
asset_values.index = pd.to_datetime(asset_values.index)
asset_values

top_movers = {}
top_movers['ticker'] = []
top_movers['latest_price'] = []
top_movers['7d_change'] = []

ranking=2

for column in asset_values.columns:
    if column == 'Portfolio Value':
        continue
    top_movers['ticker'].append(column)
    top_movers['latest_price'].append(asset_values[column].iloc[-1])
    top_movers['7d_change'].append(round(
        (asset_values[column].iloc[-1] - asset_values[column].iloc[-7])/asset_values[column].iloc[-7] * 100, 2))

df_top_movers = pd.DataFrame(top_movers).sort_values(by="7d_change", ascending=False)

top_movers = {
    'ticker': list(df_top_movers.iloc[:ranking]['ticker'].values),
    '7d_change': list(df_top_movers.iloc[:ranking]['7d_change'].values),
    'latest_price': list(df_top_movers.iloc[:ranking]['latest_price'].values)
}

print(top_movers)

[*********************100%***********************]  5 of 5 completed

{'ticker': ['SPEM', 'IEFA'], '7d_change': [np.float64(3.42), np.float64(2.64)], 'latest_price': [np.float64(2456.301465804235), np.float64(2631.2878328896104)]}
